## Ustvarite model logistične regresije - Lekcija 4

![Infografika: Logistična vs. linearna regresija](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Predhodni kviz](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Uvod

V tej zadnji lekciji o regresiji, eni izmed osnovnih *klasičnih* tehnik strojnega učenja, si bomo ogledali logistično regresijo. To tehniko bi uporabili za odkrivanje vzorcev za napovedovanje binarnih kategorij. Ali je ta sladkarija čokolada ali ne? Ali je ta bolezen nalezljiva ali ne? Ali bo ta stranka izbrala ta izdelek ali ne?

V tej lekciji boste spoznali:

-   Tehnike za logistično regresijo

✅ Poglobite svoje razumevanje dela s to vrsto regresije v tem [učnem modulu](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Predpogoji

Ker smo že delali s podatki o bučah, smo zdaj dovolj seznanjeni z njimi, da ugotovimo, da obstaja ena binarna kategorija, s katero lahko delamo: `Barva`.

Zgradimo model logistične regresije, da napovemo, glede na nekatere spremenljivke, *kakšne barve bo določena buča verjetno* (oranžna 🎃 ali bela 👻).

> Zakaj govorimo o binarni klasifikaciji v lekciji, ki se ukvarja z regresijo? Izključno zaradi jezikovne priročnosti, saj je logistična regresija [pravzaprav metoda klasifikacije](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), čeprav temelji na linearni metodi. Več o drugih načinih klasifikacije podatkov boste izvedeli v naslednji skupini lekcij.

Za to lekcijo bomo potrebovali naslednje pakete:

-   `tidyverse`: [Tidyverse](https://www.tidyverse.org/) je [zbirka paketov za R](https://www.tidyverse.org/packages), zasnovana za hitrejše, lažje in bolj zabavno delo z podatki!

-   `tidymodels`: [Tidymodels](https://www.tidymodels.org/) je okvir [zbirke paketov](https://www.tidymodels.org/packages/) za modeliranje in strojno učenje.

-   `janitor`: Paket [janitor](https://github.com/sfirke/janitor) ponuja preprosta orodja za pregledovanje in čiščenje umazanih podatkov.

-   `ggbeeswarm`: Paket [ggbeeswarm](https://github.com/eclarke/ggbeeswarm) omogoča ustvarjanje grafov v slogu "beeswarm" z uporabo ggplot2.

Namestite jih lahko z ukazom:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternativno, spodnji skript preveri, ali imate potrebne pakete za dokončanje tega modula, in jih namesti, če manjkajo.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Določite vprašanje**

Za naše namene bomo to izrazili kot binarno: 'Bela' ali 'Ne bela'. V našem naboru podatkov obstaja tudi kategorija 'črtasta', vendar je primerov te kategorije malo, zato je ne bomo uporabili. Kategorija tako ali tako izgine, ko odstranimo ničelne vrednosti iz nabora podatkov.

> 🎃 Zabavno dejstvo: bele buče včasih imenujemo 'duhove' buče. Niso zelo enostavne za rezljanje, zato niso tako priljubljene kot oranžne, vendar so videti kul! Tako bi lahko naše vprašanje preoblikovali tudi kot: 'Duh' ali 'Ne duh'. 👻

## **O logistični regresiji**

Logistična regresija se od linearne regresije, o kateri ste se že učili, razlikuje v nekaj pomembnih vidikih.

#### **Binarna klasifikacija**

Logistična regresija ne ponuja enakih funkcij kot linearna regresija. Prva ponuja napoved o `binarni kategoriji` ("oranžna ali ne oranžna"), medtem ko je druga sposobna napovedovati `neprekinjene vrednosti`, na primer glede na izvor buče in čas žetve, *koliko se bo njena cena zvišala*.

![Infografika avtorja Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Druge klasifikacije

Obstajajo tudi druge vrste logistične regresije, vključno z multinomno in ordinalno:

- **Multinomna**, ki vključuje več kot eno kategorijo - "Oranžna, Bela in Črtasta".

- **Ordinalna**, ki vključuje urejene kategorije, uporabna, če želimo logično urediti naše rezultate, na primer naše buče, ki so razvrščene po končnem številu velikosti (mini,sm,med,lg,xl,xxl).

![Multinomna vs ordinalna regresija](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Spremenljivke NI NUJNO, da so povezane**

Se spomnite, kako je linearna regresija bolje delovala z bolj povezanimi spremenljivkami? Logistična regresija je nasprotje - spremenljivke ni nujno, da se ujemajo. To ustreza tem podatkom, ki imajo nekoliko šibke korelacije.

#### **Potrebujete veliko čistih podatkov**

Logistična regresija bo dala natančnejše rezultate, če uporabite več podatkov; naš majhen nabor podatkov ni optimalen za to nalogo, zato imejte to v mislih.

✅ Razmislite o vrstah podatkov, ki bi bile primerne za logistično regresijo.

## Naloga - uredite podatke

Najprej nekoliko očistite podatke, odstranite ničelne vrednosti in izberite le nekatere stolpce:

1. Dodajte naslednjo kodo:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Vedno lahko pogledate svoj novi podatkovni okvir z uporabo funkcije [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html), kot je prikazano spodaj:


In [ ]:
pumpkins_select %>% 
  glimpse()


Potrdimo, da bomo dejansko reševali problem binarne klasifikacije:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Vizualizacija - kategorni graf
Do sedaj ste znova naložili podatke o bučah in jih očistili, da ste ohranili podatkovni niz, ki vsebuje nekaj spremenljivk, vključno z Barvo. Vizualizirajmo podatkovni okvir v zvezku z uporabo knjižnice ggplot.

Knjižnica ggplot ponuja nekaj odličnih načinov za vizualizacijo vaših podatkov. Na primer, lahko primerjate porazdelitve podatkov za vsako Sorto in Barvo v kategorni graf.

1. Ustvarite takšen graf z uporabo funkcije geombar, pri čemer uporabite naše podatke o bučah in določite barvno preslikavo za vsako kategorijo buč (oranžna ali bela):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Z opazovanjem podatkov lahko vidite, kako so podatki o barvi povezani z vrsto.

✅ Glede na ta kategorni graf, katere zanimive raziskave si lahko predstavljate?


### Predobdelava podatkov: kodiranje značilnosti

Naš nabor podatkov o bučah vsebuje nize kot vrednosti za vse stolpce. Delo s kategorialnimi podatki je za ljudi intuitivno, za računalnike pa ne. Algoritmi strojnega učenja delujejo dobro s številkami. Zato je kodiranje zelo pomemben korak v fazi predobdelave podatkov, saj nam omogoča, da kategorialne podatke pretvorimo v številske, ne da bi pri tem izgubili informacije. Dobro kodiranje vodi do gradnje dobrega modela.

Za kodiranje značilnosti obstajata dve glavni vrsti kodirnikov:

1. **Ordinalni kodirnik**: Ta je primeren za ordinalne spremenljivke, torej kategorialne spremenljivke, kjer podatki sledijo logičnemu vrstnemu redu, kot je stolpec `item_size` v našem naboru podatkov. Ustvari preslikavo, kjer je vsaka kategorija predstavljena s številko, ki ustreza vrstnemu redu kategorije v stolpcu.

2. **Kategorialni kodirnik**: Ta je primeren za nominalne spremenljivke, torej kategorialne spremenljivke, kjer podatki ne sledijo logičnemu vrstnemu redu, kot so vse značilnosti, ki niso `item_size` v našem naboru podatkov. Gre za kodiranje "ena-vroča" (one-hot encoding), kar pomeni, da je vsaka kategorija predstavljena z binarnim stolpcem: kodirana spremenljivka je enaka 1, če buča pripada tej sorti, in 0 sicer.

Tidymodels ponuja še eno uporabno knjižnico: [recipes](https://recipes.tidymodels.org/) - knjižnico za predobdelavo podatkov. Določili bomo `recipe`, ki opredeljuje, da je treba vse stolpce napovedovalcev zakodirati v nabor celih števil, nato pa ga `prep`-irati, da ocenimo potrebne količine in statistike za posamezne operacije, in na koncu `bake`, da uporabimo izračune na novih podatkih.

> Običajno se recipes uporablja kot predprocesor za modeliranje, kjer določa, kateri koraki naj se uporabijo na naboru podatkov, da ga pripravimo za modeliranje. V tem primeru je **zelo priporočljivo**, da uporabite `workflow()` namesto ročnega ocenjevanja recepta z uporabo prep in bake. Vse to bomo videli v kratkem.
>
> Zaenkrat pa uporabljamo recipes + prep + bake, da določimo, kateri koraki naj se uporabijo na naboru podatkov, da ga pripravimo za analizo podatkov, in nato izvlečemo predobdelane podatke z uporabljenimi koraki.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Kakšne so prednosti uporabe ordinalnega kodirnika za stolpec Velikost artikla?

### Analizirajte odnose med spremenljivkami

Zdaj, ko smo predhodno obdelali naše podatke, lahko analiziramo odnose med značilnostmi in oznako, da dobimo predstavo o tem, kako dobro bo model lahko napovedal oznako glede na značilnosti. Najboljši način za izvedbo takšne analize je vizualizacija podatkov. 
Ponovno bomo uporabili funkcijo ggplot geom_boxplot_, da vizualiziramo odnose med Velikostjo artikla, Sorto in Barvo v kategorijskem grafu. Za boljšo vizualizacijo podatkov bomo uporabili kodiran stolpec Velikost artikla in nekodiran stolpec Sorta.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Uporaba swarm plota

Ker je Barva binarna kategorija (Bela ali Ne), zahteva 'poseben pristop [k vizualizaciji](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)'.

Poskusite uporabiti `swarm plot`, da prikažete porazdelitev barve glede na velikost predmeta.

Uporabili bomo [paket ggbeeswarm](https://github.com/eclarke/ggbeeswarm), ki ponuja metode za ustvarjanje grafov v slogu beeswarm z uporabo ggplot2. Beeswarm grafi so način prikaza točk, ki bi se sicer prekrivale, tako da so razporejene ena poleg druge.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Zdaj, ko imamo predstavo o razmerju med binarnimi kategorijami barve in večjo skupino velikosti, raziščimo logistično regresijo za določitev verjetne barve določene buče.

## Ustvarite svoj model

Izberite spremenljivke, ki jih želite uporabiti v svojem klasifikacijskem modelu, in razdelite podatke na učne in testne sklope. [rsample](https://rsample.tidymodels.org/), paket v Tidymodels, zagotavlja infrastrukturo za učinkovito razdeljevanje podatkov in ponovno vzorčenje:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Zdaj smo pripravljeni, da model naučimo tako, da prilagodimo učne značilnosti učni oznaki (barvi).

Začeli bomo z ustvarjanjem recepta, ki določa korake predobdelave, ki jih je treba izvesti na naših podatkih, da jih pripravimo za modeliranje, tj. kodiranje kategornih spremenljivk v niz celih števil. Tako kot `baked_pumpkins`, ustvarimo `pumpkins_recipe`, vendar ga ne `prep` in `bake`, saj bo vključen v delovni tok, kar boste videli v le nekaj korakih.

Obstaja kar nekaj načinov za določitev logistične regresije v Tidymodels. Oglejte si `?logistic_reg()`. Za zdaj bomo določili model logistične regresije prek privzetega pogona `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Zdaj, ko imamo recept in specifikacijo modela, moramo najti način, kako ju združiti v objekt, ki bo najprej predobdelal podatke (v ozadju uporabil funkciji prep + bake), nato prilagodil model na predobdelane podatke in omogočil tudi morebitne aktivnosti po obdelavi.

V Tidymodels se ta priročen objekt imenuje [`workflow`](https://workflows.tidymodels.org/) in priročno združuje vaše komponente modeliranja.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Ko je potek dela *določen*, lahko model `usposobimo` z uporabo funkcije [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Potek dela bo ocenil recept in predhodno obdelal podatke pred usposabljanjem, zato tega ne bo treba ročno narediti z uporabo funkcij prep in bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Model izpiše koeficiente, pridobljene med usposabljanjem.

Zdaj, ko smo model usposobili z uporabo učnih podatkov, lahko naredimo napovedi na testnih podatkih z uporabo [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Začnimo z uporabo modela za napovedovanje oznak za naš testni niz in verjetnosti za vsako oznako. Ko je verjetnost večja od 0.5, je napovedan razred `WHITE`, sicer `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Zelo lepo! To ponuja nekaj več vpogleda v delovanje logistične regresije.

### Boljše razumevanje prek matrike zmede

Primerjanje vsake napovedi z ustrezno "resnično vrednostjo" ni zelo učinkovit način za določanje, kako dobro model napoveduje. Na srečo ima Tidymodels še nekaj trikov v rokavu: [`yardstick`](https://yardstick.tidymodels.org/) - paket, ki se uporablja za merjenje učinkovitosti modelov z uporabo metrik uspešnosti.

Ena od metrik uspešnosti, povezanih s klasifikacijskimi problemi, je [`matrika zmede`](https://wikipedia.org/wiki/Confusion_matrix). Matrika zmede opisuje, kako dobro klasifikacijski model deluje. Matrika zmede prikazuje, koliko primerov v vsakem razredu je model pravilno klasificiral. V našem primeru bo pokazala, koliko oranžnih buč je bilo klasificiranih kot oranžne in koliko belih buč kot bele; matrika zmede prav tako prikazuje, koliko jih je bilo klasificiranih v **napačne** kategorije.

Funkcija [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) iz paketa yardstick izračuna to križno tabelo opazovanih in napovedanih razredov.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Poglejmo, kako interpretirati matriko zmede. Naš model mora razvrstiti buče v dve binarni kategoriji, kategorijo `bela` in kategorijo `ne-bela`.

-   Če vaš model napove, da je buča bela, in ta v resnici spada v kategorijo 'bela', temu pravimo `prava pozitivna napoved` (true positive), kar je prikazano s številom v zgornjem levem kotu.

-   Če vaš model napove, da buča ni bela, in ta v resnici spada v kategorijo 'bela', temu pravimo `napačna negativna napoved` (false negative), kar je prikazano s številom v spodnjem levem kotu.

-   Če vaš model napove, da je buča bela, in ta v resnici spada v kategorijo 'ne-bela', temu pravimo `napačna pozitivna napoved` (false positive), kar je prikazano s številom v zgornjem desnem kotu.

-   Če vaš model napove, da buča ni bela, in ta v resnici spada v kategorijo 'ne-bela', temu pravimo `prava negativna napoved` (true negative), kar je prikazano s številom v spodnjem desnem kotu.

| Resnica |
|:-------:|

|               |        |       |
|---------------|--------|-------|
| **Napovedano** | BELA   | ORANŽNA |
| BELA          | TP     | FP    |
| ORANŽNA       | FN     | TN    |

Kot ste morda uganili, je zaželeno imeti večje število pravih pozitivnih in pravih negativnih napovedi ter manjše število napačnih pozitivnih in napačnih negativnih napovedi, saj to pomeni, da model deluje bolje.

Matrika zmede je koristna, saj omogoča izračun drugih metrik, ki nam pomagajo bolje oceniti uspešnost klasifikacijskega modela. Poglejmo si nekatere od njih:

🎓 Natančnost (Precision): `TP/(TP + FP)` je definirana kot delež napovedanih pozitivnih primerov, ki so dejansko pozitivni. Imenujemo jo tudi [pozitivna napovedna vrednost](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Priklic (Recall): `TP/(TP + FN)` je definiran kot delež pozitivnih rezultatov glede na število vzorcev, ki so dejansko pozitivni. Imenujemo ga tudi `občutljivost` (sensitivity).

🎓 Specifičnost (Specificity): `TN/(TN + FP)` je definirana kot delež negativnih rezultatov glede na število vzorcev, ki so dejansko negativni.

🎓 Točnost (Accuracy): `TP + TN/(TP + TN + FP + FN)` je odstotek oznak, ki so bile pravilno napovedane za vzorec.

🎓 F-metrika (F Measure): Tehtano povprečje natančnosti in priklica, kjer je najboljša vrednost 1, najslabša pa 0.

Izračunajmo te metrike!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Vizualizirajmo ROC krivuljo tega modela

Naredimo še eno vizualizacijo, da si ogledamo tako imenovano [`ROC krivuljo`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC krivulje se pogosto uporabljajo za prikaz rezultatov klasifikatorja glede na njegove prave in lažne pozitivne zadetke. ROC krivulje običajno prikazujejo `True Positive Rate`/občutljivost na Y osi in `False Positive Rate`/1-specifičnost na X osi. Zato sta strmina krivulje in prostor med sredinsko črto ter krivuljo pomembna: želite krivuljo, ki se hitro dvigne in preseže črto. V našem primeru se začne z lažnimi pozitivnimi zadetki, nato pa se črta pravilno dvigne in preseže.

Na koncu uporabimo `yardstick::roc_auc()` za izračun dejanskega območja pod krivuljo (Area Under the Curve). Eden od načinov interpretacije AUC je kot verjetnost, da model naključno pozitivni primer razvrsti višje kot naključno negativni primer.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Rezultat je približno `0.975`. Ker se AUC giblje med 0 in 1, si želite visok rezultat, saj bo model, ki je 100 % natančen v svojih napovedih, imel AUC vrednost 1; v tem primeru je model *kar dober*.

V prihodnjih lekcijah o klasifikacijah boste spoznali, kako izboljšati rezultate svojega modela (na primer obravnavanje neuravnoteženih podatkov v tem primeru).

## 🚀Izziv

Logistična regresija ponuja še veliko več! Najboljši način za učenje je eksperimentiranje. Poiščite podatkovni niz, ki je primeren za tovrstno analizo, in zgradite model z njim. Kaj ste se naučili? namig: poskusite [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) za zanimive podatkovne nize.

## Pregled in samostojno učenje

Preberite prvih nekaj strani [tega dokumenta iz Stanforda](https://web.stanford.edu/~jurafsky/slp3/5.pdf) o praktični uporabi logistične regresije. Razmislite o nalogah, ki so bolj primerne za eno ali drugo vrsto regresijskih nalog, ki smo jih preučevali do zdaj. Kaj bi delovalo najbolje?



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
